In [1]:
import os
import cv2
import json
import copy
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt
from inference.infer import PoseDetector, ActionClassifier
from inference.rules import Rules
import pandas as pd

In [2]:
os.chdir('../')

In [3]:
os.listdir('./data_folder/labels/pro_jsons/')

['0.json',
 '1.json',
 '10.json',
 '11.json',
 '12.json',
 '13.json',
 '14.json',
 '15.json',
 '16.json',
 '17.json',
 '2.json',
 '3.json',
 '4.json',
 '5.json',
 '6.json',
 '7.json',
 '8.json',
 '9.json']

In [5]:
os.getcwd()

'D:\\ML\\ai_golf\\teamproject_golf'

In [9]:
rules = []
for i in os.listdir('data_folder/labels/pro_jsons/'):
    with open(os.path.join('./data_folder/labels/pro_jsons/', i), 'rb') as r:
        rules.append(json.load(r))

In [10]:
rules[0]

{'address': {'어깨선각도': 5.63, '양손위치': 45.5, '스탠스': 62.01, '상체기울임정도': 174.84},
 'backswing': {'왼쪽어깨회전': 61.68, '왼팔펴짐각도': 149.95},
 'top': {'하체고정': 42.59, '오른쪽다리펴짐각도': 179.0, '스웨이체크': 43.74},
 'impact': {'행잉백': 6.19,
  '오른쪽K라인각도': 146.12,
  '왼팔펴짐각도': 171.17,
  '오른팔펴짐각도': 167.09},
 'follow': {'왼쪽라인': 4.83, '치킨윙': 175.9, '체중전진이동': 63.5, '오른쪽다리펴짐각도': 159.91}}

In [26]:
address_df = pd.DataFrame()
for rule in rules:
    if len(address_df) == 0:
        address_df = pd.DataFrame(data=[list(rule['address'].values())],
                                 columns=list(rule['address'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['address'].values())],
                                 columns=list(rule['address'].keys()))
        address_df = pd.concat([address_df, new])

address_df.reset_index(inplace=True,drop=True)
address_df.describe()

,어깨선각도,양손위치,스탠스,상체기울임정도
count,18.000000,18.000000,18.000000,18.000000
mean,10.718889,51.009444,57.922778,181.328333
std,3.403249,5.439118,5.706392,9.665421
min,3.270000,41.460000,47.740000,164.810000
25%,8.425000,46.317500,54.382500,175.150000
50%,11.330000,52.655000,56.240000,180.440000
75%,12.870000,54.740000,61.897500,184.575000
max,15.210000,59.120000,71.500000,199.090000


In [39]:
backswing_df = pd.DataFrame()
for rule in rules:
    if len(backswing_df) == 0:
        backswing_df = pd.DataFrame(data=[list(rule['backswing'].values())],
                                 columns=list(rule['backswing'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['backswing'].values())],
                                 columns=list(rule['backswing'].keys()))
        backswing_df = pd.concat([backswing_df, new])

backswing_df.reset_index(inplace=True,drop=True)
backswing_df.describe()

,왼쪽어깨회전,왼팔펴짐각도
count,18.000000,18.000000
mean,47.181111,147.475556
std,8.981905,18.825317
min,30.990000,93.130000
25%,41.612500,142.365000
50%,47.475000,152.360000
75%,54.035000,157.572500
max,61.680000,172.190000


In [35]:
top_df = pd.DataFrame()
for rule in rules:
    if len(top_df) == 0:
        top_df = pd.DataFrame(data=[list(rule['top'].values())],
                                 columns=list(rule['top'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['top'].values())],
                                 columns=list(rule['top'].keys()))
        top_df = pd.concat([top_df, new])

top_df.reset_index(inplace=True,drop=True)
top_df.describe()

,하체고정,오른쪽다리펴짐각도,스웨이체크
count,18.000000,18.000000,18.000000
mean,38.784444,173.318333,27.542222
std,8.348269,5.564564,10.233604
min,17.500000,159.000000,8.360000
25%,36.385000,170.775000,23.965000
50%,41.870000,174.220000,29.945000
75%,44.535000,178.157500,35.400000
max,48.580000,179.200000,43.740000


In [37]:
impact_df = pd.DataFrame()
for rule in rules:
    if len(impact_df) == 0:
        impact_df = pd.DataFrame(data=[list(rule['impact'].values())],
                                 columns=list(rule['impact'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['impact'].values())],
                                 columns=list(rule['impact'].keys()))
        impact_df = pd.concat([impact_df, new])

impact_df.reset_index(inplace=True,drop=True)
impact_df.describe()

,행잉백,오른쪽K라인각도,왼팔펴짐각도,오른팔펴짐각도
count,18.000000,18.000000,18.000000,18.000000
mean,8.542778,148.590000,172.623333,159.403333
std,3.430462,11.429555,6.064848,10.579996
min,2.470000,131.660000,158.980000,123.940000
25%,6.965000,141.125000,169.255000,155.962500
50%,7.925000,145.560000,174.880000,160.930000
75%,9.487500,158.457500,176.642500,165.650000
max,16.580000,168.170000,179.600000,172.490000


In [38]:
follow_df = pd.DataFrame()
for rule in rules:
    if len(follow_df) == 0:
        follow_df = pd.DataFrame(data=[list(rule['follow'].values())],
                                 columns=list(rule['follow'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['follow'].values())],
                                 columns=list(rule['follow'].keys()))
        follow_df = pd.concat([follow_df, new])

follow_df.reset_index(inplace=True,drop=True)
follow_df.describe()

,왼쪽라인,치킨윙,체중전진이동,오른쪽다리펴짐각도
count,18.000000,18.000000,18.000000,18.000000
mean,6.010556,163.022778,62.194444,157.693333
std,3.409099,25.749572,8.352250,11.935176
min,0.170000,63.860000,48.710000,138.030000
25%,4.365000,161.320000,55.517500,144.677500
50%,5.875000,169.955000,62.590000,160.165000
75%,8.592500,174.217500,69.417500,167.555000
max,12.210000,177.230000,74.740000,174.600000


In [ ]:
top_df = pd.DataFrame()
for rule in rules:
    if len(back_df) == 0:
        back_df = pd.DataFrame(data=[list(rule['backswing'].values())],
                                 columns=list(rule['backswing'].keys()))
    else:
        new = pd.DataFrame(data=[list(rule['backswing'].values())],
                                 columns=list(rule['backswing'].keys()))
        back_df = pd.concat([back_df, new])

back_df.reset_index(inplace=True,drop=True)
back_df.describe()

In [ ]:
a = list(esti_rule['address'].values())

b = list(esti_rule['address'].keys())

pd.DataFrame(data=[a], columns=b)

In [5]:
# ----------------------------------------------------------

rule = Rules()
with open('data_folder/parameter/params.json', 'r') as r:
    params = json.load(r)

In [7]:
actions = params.keys()

for idx, file in enumerate(os.listdir('data_folder/src/pro/')[14:]):
    idx += 14
    src_dir = os.path.join('data_folder/src/pro/', file)
    print(src_dir)
    ac = ActionClassifier()
    pose = PoseDetector()
    for action in actions:
        video = cv2.VideoCapture(src_dir)
        while video.isOpened():
            read_ok, frame = video.read()
            if not read_ok:
#                 print('TRK Complete!')
                break
            cnt = video.get(cv2.CAP_PROP_POS_FRAMES)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, dsize=(0, 0), fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
            pose_obj = pose.trk_process(image=frame)
            pose.get_coordis_3d(pose_obj)
            landmarked_img = pose.drawing_with_pose(frame, pose_obj)
            ac.update_information(action=action,
                                parameter=params,
                                pose_coordis=pose.coordinates,
                                num_frame=cnt, image=landmarked_img)
    with open('data_folder/parameter/label_rule_value.json', 'rb') as r:
        label_rule = json.load(r)
    esti_rule = copy.deepcopy(label_rule)
    for action in ac.esti_inform:
        esti_rule = rule.handle(action=action, 
                        value=esti_rule, 
                        coordis=ac.esti_inform[action]['coordinate'])
    buf = './data_folder/labels/pro_jsons/' + str(idx) + '.json'
    with open(buf, 'w',encoding='utf-8') as f:
        json.dump(esti_rule, f, indent=4, ensure_ascii=False)
    del(esti_rule)
    print('-' * 20)

data_folder/src/pro/타이거우즈 드라이버 시퀀스 !!.mp4
--------------------
data_folder/src/pro/타이거우즈 스윙 슬로우모션 100번돌려보기.mp4
--------------------
data_folder/src/pro/프로골퍼 🏌️_♀️ 드라이버 스윙, 골프 스윙 , #shorts ❤️ 임선수 🌿  Im Ji Dong (12).mp4
--------------------
data_folder/src/pro/프로골퍼 🏌️_♀️ 드라이버 스윙, 골프 스윙 , #shorts 💟 김진주 💎 Kim Jin Joo (2).mp4
--------------------
data_folder/src/pro/환상적이야! 로리맥길로이 와 함께 드라이버스윙 #골프스윙 #드라이버스윙 #로리맥킬로이 #로리맥길로이.mp4


AttributeError: 'NoneType' object has no attribute 'landmark'

In [ ]:
os.lis

In [6]:
os.listdir('./data_folder/src/pro/')[14:]

['타이거우즈 드라이버 시퀀스 !!.mp4',
 '타이거우즈 스윙 슬로우모션 100번돌려보기.mp4',
 '프로골퍼 🏌️_♀️ 드라이버 스윙, 골프 스윙 , #shorts ❤️ 임선수 🌿  Im Ji Dong (12).mp4',
 '프로골퍼 🏌️_♀️ 드라이버 스윙, 골프 스윙 , #shorts 💟 김진주 💎 Kim Jin Joo (2).mp4',
 '환상적이야! 로리맥길로이 와 함께 드라이버스윙 #골프스윙 #드라이버스윙 #로리맥킬로이 #로리맥길로이.mp4',
 '💥 오늘도 저에게 많은 자극 주시는 임민국 프로님 아이언 스윙+슬로우모션 #shorts.mp4']

In [11]:
label_rule

{'address': {'어깨선각도': 10.96, '양손위치': 55.51, '스탠스': 53.11, '상체기울임정도': 177.64},
 'backswing': {'왼쪽어깨회전': 54.13, '왼팔펴짐각도': 157.49},
 'top': {'하체고정': 47.89, '오른쪽다리펴짐각도': 175.82, '스웨이체크': 40.04},
 'impact': {'행잉백': 7.56,
  '오른쪽K라인각도': 138.23,
  '왼팔펴짐각도': 176.85,
  '오른팔펴짐각도': 152.05},
 'follow': {'왼쪽라인': 5.17, '치킨윙': 168.08, '체중전진이동': 68.82, '오른쪽다리펴짐각도': 174.71}}

In [9]:
esti_rule

{'address': {'어깨선각도': 7.24, '양손위치': 48.77, '스탠스': 55.08, '상체기울임정도': 164.81},
 'backswing': {'왼쪽어깨회전': 48.34, '왼팔펴짐각도': 163.81},
 'top': {'하체고정': 45.01, '오른쪽다리펴짐각도': 174.56, '스웨이체크': 35.07},
 'impact': {'행잉백': 7.1,
  '오른쪽K라인각도': 157.61,
  '왼팔펴짐각도': 176.53,
  '오른팔펴짐각도': 160.01},
 'follow': {'왼쪽라인': 6.46, '치킨윙': 177.23, '체중전진이동': 58.53, '오른쪽다리펴짐각도': 144.28}}